# Preparativos

In [24]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download zaheenhamidani/ultimate-spotify-tracks-db

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [6]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

urlTrain  = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures_train.csv'
urlFull = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures.csv'
# el train que nos dieron los profes
#df_train_original= pd.read_csv('/content/SpotifyFeatures_train.csv')
df_train_original = pd.read_csv(urlTrain)
df_train = df_train_original.copy()
# el dataset nuestro
df_full_original = pd.read_csv(urlFull)
df_full = df_full_original.copy()


# Exploracion de datos

In [7]:
df_train.head()

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,107360,Opera,Richard Wagner,Gotterdammerung (Twilight of the Gods): Prolog...,32pnRCr5u3nzO55MgoEJwV,6,0.89900,0.147,131787,0.016,0.947000,B,0.0889,-24.317,Major,0.0380,137.591,4/4,0.0292
1,165017,Reggaeton,Ronald El Killa,Esta Pena,3uMQ7q5oU0VgYl9SyCF2MR,27,0.00613,0.857,255134,0.599,0.000085,B,0.2090,-5.677,Major,0.0951,96.022,4/4,0.5860
2,141044,Indie,LP,Recovery,1a5P7IvLUhRhLnnbkev3PI,59,0.27400,0.518,234905,0.545,0.000023,A#,0.1830,-7.893,Major,0.0385,75.060,4/4,0.1620
3,141709,Indie,The National,Carin at the Liquor Store,4maI5FwVIzjrErduPjjZLO,53,0.59400,0.540,213733,0.615,0.803000,A#,0.1200,-9.264,Major,0.0340,93.170,3/4,0.2390
4,106321,Opera,Renata Tebaldi,Giordano: Andrea Chenier: Come un bel di di ma...,5VEVnqQu6cFewcbWfbkUeE,8,0.95500,0.363,170240,0.196,0.000115,C#,0.6840,-15.675,Minor,0.0452,63.186,4/4,0.2510


In [27]:
#df_full.head()

In [28]:
df_train.columns

Index(['Unnamed: 0', 'genre', 'artist_name', 'track_name', 'track_id',
       'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [29]:
df_train.dtypes

Unnamed: 0            int64
genre                object
artist_name          object
track_name           object
track_id             object
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

# Limpieza de datos

In [3]:
if 'Unnamed: 0' in df_train.columns:
    df_train = df_train.drop(['Unnamed: 0'], axis=1)
else:
    print('no esta')  


no esta


In [3]:

indexNamesComedy = df_train[ df_train['genre'] == 'Comedy' ].index
df_train.drop(indexNamesComedy , inplace=True)

In [32]:
df_train['genre'].value_counts()

Soundtrack          7758
Jazz                7634
Indie               7603
Children’s Music    7554
Pop                 7532
Electronic          7500
Rock                7455
Hip-Hop             7422
Folk                7406
Classical           7395
Rap                 7367
Alternative         7342
Soul                7280
R&B                 7242
World               7211
Blues               7187
Ska                 7154
Reggaeton           7139
Anime               7131
Reggae              7019
Dance               6950
Country             6861
Opera               6612
Movie               6261
Children's Music    4298
A Capella             96
Name: genre, dtype: int64

Vemos que la columna Children esta repetida

In [33]:
indexNamesChildren = df_train[ df_train['genre'] == "Children's Music" ].index
df_train.drop(indexNamesChildren , inplace=True)

### Busqueda de datos repetidos

In [34]:

if df_train.duplicated().sum() == 0:
  print('no hay datos duplicados')
else:
  df_train.drop_duplicates


no hay datos duplicados


### Busqueda de datos faltantes

In [35]:
#columnas con datos faltantes
cols_with_missing = [col for col in df_train.columns if df_train[col].isnull().any()] 
faltanDatos = True
if len(cols_with_missing)==0:
  faltanDatos= False
  print('no faltan valores') 

no faltan valores


In [36]:
# saco las filas correspondientes
if faltanDatos == True:
    print(cols_with_missing)
    df_train.dropna(axis=0, inplace=True)


# Variables Categoricas

In [37]:
# Variable categoricas
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['genre', 'artist_name', 'track_name', 'track_id', 'key', 'mode', 'time_signature']


In [38]:
list_of_timeSignature = df_train['time_signature'].unique()
print(list_of_timeSignature)

['4/4' '3/4' '5/4' '1/4' '0/4']


In [39]:
df_train['time_signature'].value_counts()

4/4    153286
3/4     15927
5/4      3294
1/4      1598
0/4         6
Name: time_signature, dtype: int64

Decidimos no utilizar Time_signature como feature por varios motivos


*   En musica no existe compaces de 1/4 ni de 0/4 entre ambos suman 1656 valores lo que nos parecio un numero considerable
*   En una cancion puede haber multiples cambios de compaces, en la musica progresiva es algo muy comun

*   La mayoria de la musica se encuentra en 4/4 podemos ver que hay 156851 valores que considerablemente mayor a lo suma de todas las otras categorias esto introduceria un gran sesgo en nuestro modelo 










In [40]:
df_train = df_train.drop(['time_signature'], axis=1)

In [41]:
# variables categoricas con baja cardinalidad
categorical_cols = [cname for cname in df_train.columns if
                    df_train[cname].nunique() < 14 and 
                    df_train[cname].dtype == "object"]
print(categorical_cols)

['key', 'mode']


Quedan descartadas las variables categoricas: 

*   Time_signature por los motivos comentados anteriormente
*   track_id: no hay ninguna razon para utilizarla
*   el nombre del artista: son demasiadas variables de momento las sacamos pero es cierto que es un feature interesente 
*   track_name: todos sus valores son distintos 







In [42]:
list_of_modes = df_train['mode'].unique()
print(list_of_modes)

['Major' 'Minor']


In [43]:
list_of_keys = df_train['key'].unique()
print(list_of_keys)


['B' 'A#' 'C#' 'C' 'F#' 'E' 'A' 'D' 'G' 'F' 'G#' 'D#']


### Preprocesado de datos

In [12]:
# Funcion para el preprocesamiento de datos
from sklearn.model_selection import train_test_split

def preprocesado(df):

    if 'Unnamed: 0' in df.columns:
        df = df.drop(['Unnamed: 0'], axis=1)
    else:
        print('no esta')  

    df = df.drop(['time_signature','track_id','artist_name','track_name'], axis=1)
    indexNamesComedy = df[ df['genre'] == 'Comedy' ].index
    indexNamesChildren = df[ df['genre'] == "Children's Music" ].index
    df.drop(indexNamesComedy , inplace=True)
    df.drop(indexNamesChildren , inplace=True)
    
    cols_with_missing = [col for col in df.columns if df[col].isnull().any()] 
    faltanDatos = True
    if len(cols_with_missing)==0:
        faltanDatos= False
        print('no faltan valores') 
    if faltanDatos == True:
        print(cols_with_missing)
        df.dropna(axis=0, inplace=True)

    X = df.drop(columns=['genre'])
    y = df['genre']
    

    return X,y 


In [13]:
X,y = preprocesado(df_train)

no faltan valores


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,6,0.89900,0.147,131787,0.01600,0.947000,B,0.0889,-24.317,Major,0.0380,137.591,0.0292
1,27,0.00613,0.857,255134,0.59900,0.000085,B,0.2090,-5.677,Major,0.0951,96.022,0.5860
2,59,0.27400,0.518,234905,0.54500,0.000023,A#,0.1830,-7.893,Major,0.0385,75.060,0.1620
3,53,0.59400,0.540,213733,0.61500,0.803000,A#,0.1200,-9.264,Major,0.0340,93.170,0.2390
4,8,0.95500,0.363,170240,0.19600,0.000115,C#,0.6840,-15.675,Minor,0.0452,63.186,0.2510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186174,35,0.97200,0.173,190067,0.00445,0.549000,G#,0.1060,-33.642,Major,0.0517,99.080,0.0399
186176,30,0.02240,0.698,238185,0.60800,0.018900,B,0.1030,-5.307,Minor,0.1370,75.118,0.5960
186177,72,0.00190,0.607,325502,0.81300,0.000880,E,0.3050,-5.825,Minor,0.0936,133.962,0.1950
186178,63,0.22600,0.664,254147,0.35100,0.000002,C,0.1060,-12.136,Major,0.1180,113.204,0.8160


In [45]:
X = df_train.drop(columns=['genre'])
y = df_train['genre']

In [46]:
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [52]:
X_train.select_dtypes('object')

,key,mode
101822,A#,Major
83812,D,Major
2556,C#,Major
174265,G#,Minor
136222,F#,Minor
...,...,...
102638,G,Major
162868,G,Major
126161,C,Major
185725,A,Major


In [49]:

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

print(numerical_cols)

['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']


In [50]:
#Seleccion de columnas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


# Un poco de ingenieria de datos

In [51]:
from sklearn.preprocessing import OrdinalEncoder


label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
label_X_valid[categorical_cols] = ordinal_encoder.transform(X_valid[categorical_cols])




### Viendo si la transformacion esta bien